In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('./output/test_hypothesis_evidences.csv')
test_df

,hypothesis,evidences,label
0,Women suffering from both claustrophobia and I...,"[""Exclusion Criteria:"", "" History of bilatera...",Contradiction
1,There are no conditions on mental mental healt...,"[""Inclusion Criteria:"", "" Women 18 years"", "" ...",Entailment
2,the secondary trial requires patients to be ov...,"[""Inclusion Criteria:"", "" Patients presenting...",Entailment
3,Patients currently taking part in the secondar...,"["" Participation in another clinical trial wi...",Entailment
4,HER2 + Patients in the primary trial receive t...,"[""INTERVENTION 1: "", "" Trastuzumab"", "" Parti...",Contradiction
...,...,...,...
495,alcoholic patients are excluded from the prima...,"["" Be non-smokers (no nicotine use within the...",Contradiction
496,Patients with a histologically/cytologically c...,"["" Non-small cell lung cancer"", "" Small cell...",Entailment
497,the primary trial treats one of its patient co...,"[""INTERVENTION 1: "", "" Pregabalin150"", "" Pat...",Entailment
498,after a month of TAK-228 Plus Tamoxifen treatm...,"[""Outcome Measurement: "", "" Ki67 Expression"",...",Contradiction


In [3]:
hypothesis_lst=test_df['hypothesis'].values.tolist()
len(hypothesis_lst)

500

In [4]:
evidence_lst=test_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

500

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

500

In [6]:
import json
with open('output/all_evidences.json','r') as f:
    all_evidences = json.load(f)
evidence2id=dict([(all_evidences[i],i) for i in range(len(all_evidences))])

from nltk.tokenize import word_tokenize
tokenized_corpus = [word_tokenize(doc) for doc in all_evidences]

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tokenized_corpus)

In [7]:
import numpy as np
scores=[]
for hypothesis,evidences in test_df[['hypothesis','evidences']].values:
    doc_scores = bm25.get_scores(word_tokenize(hypothesis))
    doc_scores = (doc_scores-np.min(doc_scores))/(np.max(doc_scores)-np.min(doc_scores))
    scores.append(np.mean([doc_scores[evidence2id[evi]] for evi in json.loads(evidences)]))
# scores

In [8]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for thre in np.arange(0.0, 1.0, 0.1):
    y_prob=scores
    y_pred=[1 if a>thre else 0 for a in y_prob]
    results.append([
        thre,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['threshold','AVG_PREC','F1','PREC','REC','ACC'])

,threshold,AVG_PREC,F1,PREC,REC,ACC
0,0.0,0.486008,0.622093,0.488584,0.856,0.480
1,0.1,0.486008,0.506518,0.473868,0.544,0.470
2,0.2,0.486008,0.397129,0.494048,0.332,0.496
3,0.3,0.486008,0.261128,0.505747,0.176,0.502
4,0.4,0.486008,0.103448,0.375000,0.060,0.480
5,0.5,0.486008,0.066667,0.450000,0.036,0.496
6,0.6,0.486008,0.053640,0.636364,0.028,0.506
7,0.7,0.486008,0.023529,0.600000,0.012,0.502
8,0.8,0.486008,0.015810,0.666667,0.008,0.502
9,0.9,0.486008,0.015810,0.666667,0.008,0.502
